In [1]:
# import basic libraries
import os
from os import listdir
import sys
import json
import datetime

In [2]:
# import advance libraries
from xml.etree import ElementTree
import skimage.draw
import cv2
import imgaug

In [3]:
# ROOT
ROOT = os.getcwd()
DATASET = os.path.join(ROOT,'dataset')
TRAIN = os.path.join(DATASET,'train')
VALIDATION = os.path.join(DATASET,'validation')

In [4]:
DATASET

'/home/projeto_chn_/pibic_cch_maskRCNN_test/dataset'

In [4]:
# !alias pip=/usr/bin/pip

In [3]:
!python --version

Python 3.7.17


In [5]:
import tensorflow as tf
print(tf.__version__)

1.14.0


In [5]:
# import mask rcnn libraries
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize

/home/ubuntu/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: 

<h1>change dir and download model</h1>
<p>use:</p>
<span>!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5</span>

In [7]:
# import matplotlib library
import matplotlib.pyplot as plt

# import numpy libraries
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean

In [6]:
# import keras libraries
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

# inline matplotlib
%matplotlib inline


In [24]:
import math


class CchDataset(Dataset):
    
    # load_dataset function is used to load the train and test dataset
    def load_dataset(self, DATASET, is_train=True):
        
        # we add a class that we need to classify in our case it is Damage
        # self.add_class("dataset", 1, "Damage")
        self.add_class("dataset", 1, "Femea")
        self.add_class("dataset", 2, "Macho")
        self.add_class("dataset", 3, "Linfa")

        # anotation and image source
        IMAGE_DIR =  os.path.join(DATASET,'images')
        ANNOTATIONS_DIR = os.path.join(DATASET,'annotations')
        LENGHT = listdir(IMAGE_DIR).__len__()
        OFFSET =  math.floor(LENGHT*0.8)

        # print(f'offset: {offset}')
        # is_train will be true if we our training our model and false when we are testing the model
        for index,filename in enumerate(listdir(IMAGE_DIR)):
            
            # extract image id
            image_id = filename[:-4] # used to skip last 4 chars which is '.jpg' (class_id.jpg)
            
            
            img_path = IMAGE_DIR +'/'+filename
            ann_path = ANNOTATIONS_DIR+'/'+ image_id + '.xml'

            # print(f'exist? >> {os.path.exists(img_path)} >> path_ing: {img_path}')
            # print(f'exist? >> {os.path.exists(ann_path)} >> path_ann: {ann_path}')

            # regra dos 80/20
            if not is_train:
                if index > OFFSET:
                    # print(f'passs>> {index}')
                    continue
            else:
                if index < OFFSET:
                    # print(f'passs>> {index}')
                    continue

            # using add_image function we pass image_id, image_path and ann_path so that the current
            # image is added to the dataset for training or testing

            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # function used to extract bouding boxes from annotated files
    def extract_boxes(self, filename):

        # you can see how the images are annotated we extracrt the width, height and bndbox values
        # <annotation>
        # <size>
        #       <width>640</width>
        #       <height>360</height>
        #       <depth>3</depth>
        # </size>
        # <object>
        #          <name>damage</name>
        #          <pose>Unspecified</pose>
        #          <truncated>0</truncated>
        #          <difficult>0</difficult>
        #          <bndbox>
        #                 <xmin>315</xmin>
        #                 <ymin>160</ymin>
        #                 <xmax>381</xmax>
        #                 <ymax>199</ymax>
        #          </bndbox>
        # </object>
        # </annotation>
        
        # used to parse the .xml files
        tree = ElementTree.parse(filename)
        
        # to get the root of the xml file
        root = tree.getroot()
        
        # we will append all x, y coordinated in boxes
        # for all instances of an onject
        boxes = list()
        
        # we find all attributes with name bndbox
        # bndbox will exist for each ground truth in image
        for box in root.findall('.//object'):
            name = box.find('name').text
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax, name]
            boxes.append(coors)
        
            # I have included this line to skip any un-annotated images
            if name=='Femea' or name=='Macho' or name=='Linfa':
                boxes.append(coors)

        # extract width and height of the image
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # return boxes-> list, width-> int and height-> int 
        return boxes, width, height
    
    # this function calls on the extract_boxes method and is used to load a mask for each instance in an image
    # returns a boolean mask with following dimensions width * height * instances
    def load_mask(self, image_id):
        
        # info points to the current image_id
        info = self.image_info[image_id]
        
        # we get the annotation path of image_id which is dataset_dir/annots/image_id.xml
        path = info['annotation']
        
        # we call the extract_boxes method(above) to get bndbox from .xml file
        boxes, w, h = self.extract_boxes(path)
        
        # we create len(boxes) number of masks of height 'h' and width 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        
        
        class_ids = list()
        
        # we loop over all boxes and generate masks (bndbox mask) and class id for each instance
        # masks will have rectange shape as we have used bndboxes for annotations
        # for example:  if 2.jpg have three objects we will have following masks and class_ids
        # 000000000 000000000 000001110 
        # 000011100 011100000 000001110
        # 000011100 011100000 000001110
        # 000000000 011100000 000000000
        #    1         1          1    <- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            
            
            if (box[4] == 'Level-1'):
                masks[row_s:row_e, col_s:col_e, i] = 1
                class_ids.append(self.class_names.index('Femea'))
            elif(box[4] == 'Level-2'):
                masks[row_s:row_e, col_s:col_e, i] = 2
                class_ids.append(self.class_names.index('Macho')) 
            elif(box[4] == 'Level-3'):
                masks[row_s:row_e, col_s:col_e, i] = 3
                class_ids.append(self.class_names.index('Linfa'))
           
        
        # return masks and class_ids as array
        return masks, asarray(class_ids, dtype='int32')
    
    # this functions takes the image_id and returns the path of the image
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']



<p>teste o codigo acima</p>

In [25]:
cch_data_class = CchDataset()
cch_data_class.load_dataset(DATASET,False)

exist? >> True >> path_ing: /home/projeto_chn_/pibic_cch_maskRCNN_test/dataset/images/666392d9-editada_90_B3.jpeg_quadrante_3_1.jpg
exist? >> True >> path_ing: /home/projeto_chn_/pibic_cch_maskRCNN_test/dataset/images/9f254806-editada_90_B7.jpeg_quadrante_2_1.jpg
exist? >> True >> path_ing: /home/projeto_chn_/pibic_cch_maskRCNN_test/dataset/images/660692fd-editada_90_B2.jpeg_quadrante_2_2.jpg
exist? >> True >> path_ing: /home/projeto_chn_/pibic_cch_maskRCNN_test/dataset/images/f4cf9830-editada_flipped_B21.jpeg_quadrante_4_3.jpg
exist? >> True >> path_ing: /home/projeto_chn_/pibic_cch_maskRCNN_test/dataset/images/a965b8aa-editada_180_B3.jpeg_quadrante_3_4.jpg
exist? >> True >> path_ing: /home/projeto_chn_/pibic_cch_maskRCNN_test/dataset/images/9ce02bec-editada_flipped_B22.jpeg_quadrante_2_2.jpg
exist? >> True >> path_ing: /home/projeto_chn_/pibic_cch_maskRCNN_test/dataset/images/215e4820-editada_flipped_B21.jpeg_quadrante_2_3.jpg
exist? >> True >> path_ing: /home/projeto_chn_/pibic_cch_

In [ ]:
# damage configuration class, you can change values of hyper parameters here
class CchConfig(Config):
    # name of the configuration
    NAME = "conchonilha"
    
    # damage class + background class
    NUM_CLASSES = 3 + 1
    
    # steps per epoch and minimum confidence
    STEPS_PER_EPOCH = 80
    
    # learning rate and momentum
    LEARNING_RATE=0.002
    LEARNING_MOMENTUM = 0.8
    
    # regularization penalty
    WEIGHT_DECAY = 0.0001
    
    # image size is controlled by this parameter
    IMAGE_MIN_DIM = 300
    
    # validation steps
    VALIDATION_STEPS = 50
    
    # number of Region of Interest generated per image
    Train_ROIs_Per_Image = 200
    
    # RPN Acnhor scales and ratios to find ROI
    RPN_ANCHOR_SCALES = (16, 32, 48, 64, 128)
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]


In [ ]:
# TODO CONTINUE... 
# load the train dataset
train_set = CchDataset()
train_set.load_dataset('customImages/stage-1', is_train=True)
train_set.prepare()

# load the test dataset
test_set = CchDataset()
test_set.load_dataset('customImages/stage-1', is_train=False)
test_set.prepare()

# prepare config by calling the user defined confifuration class
config = CchConfig()

# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)

# load weights mscoco model weights
weights_path = 'mask_rcnn_coco.h5'

# load the model weights
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])


# start the training of model
# you can change epochs and layers (head or all)
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')